In [ ]:
!kaggle datasets download -d vitngquang/uit-viic-v1-0-vietnamese-image-captioning

Dataset URL: https://www.kaggle.com/datasets/vitngquang/uit-viic-v1-0-vietnamese-image-captioning
License(s): other
100% 1.23G/1.23G [01:04<00:00, 23.1MB/s]
100% 1.23G/1.23G [01:04<00:00, 20.6MB/s]


In [ ]:
!unzip -q uit-viic-v1-0-vietnamese-image-captioning.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Mô hình

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
import torch
import os
import shutil

In [ ]:
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
shutil.move("kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

In [ ]:
!kaggle kernels output hatakacder/fine-tuning-blip-image-captioning-base -p /content

Output file downloaded to /content/best_model.pth
User cancelled operation


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("/content/best_model.pth", map_location=device))
model = model.to(device)

<ipython-input-7-5c6ad945de2b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/best_model.pth", map_location=device))


# Xử lý dữ liệu

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import Dataset
from PIL import Image

In [ ]:
def load_data(path):
    with open(path + 'captions.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = [line.split('\t', 1) for line in lines]
    df = pd.DataFrame(data, columns=["image_path", "caption"])

    return df

In [ ]:
dataset_path = '/content/dataset'

In [ ]:
train_path = dataset_path + '/train/'
val_path = dataset_path + '/val/'
test_path = dataset_path + '/test/'

In [ ]:
df_train = load_data(train_path)
df_val = load_data(val_path)
df_test = load_data(test_path)

In [ ]:
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.8 MB/s eta 0:00:00


In [ ]:
from pyvi import ViUtils, ViTokenizer

def text_preprocessing(text):
    text = text.lower()
    text = ViTokenizer.tokenize(text)
    text = text.replace('\n', ' ')
    text = ViUtils.remove_accents(text)
    text = text.decode("utf-8")
    text = text.replace("'", "")
    text = text.lstrip('b')
    return text

In [ ]:
df_train['caption'] = df_train['caption'].apply(lambda x: text_preprocessing(x))
df_val['caption'] = df_val['caption'].apply(lambda x: text_preprocessing(x))
df_test['caption'] = df_test['caption'].apply(lambda x: text_preprocessing(x))

KeyboardInterrupt: 

In [ ]:
df_train

In [ ]:
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)

In [ ]:
dataset_path_2 = '/content'

def load_image(image_path):
  image_path = dataset_path_2 + image_path
  return Image.open(image_path)

In [ ]:
train_dataset = train_dataset.map(lambda example: {"image": load_image(example["image_path"])}, batched=False)
train_dataset = train_dataset.remove_columns(["image_path"])
val_dataset = val_dataset.map(lambda example: {"image": load_image(example["image_path"])}, batched=False)
val_dataset = val_dataset.remove_columns(["image_path"])

In [ ]:
import torch.utils.data

class ImageCaptioningDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], text=item["caption"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

In [ ]:
train_dataset = ImageCaptioningDataset(train_dataset, processor)
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=4)
val_dataset = ImageCaptioningDataset(val_dataset, processor)
val_dataloader = torch.utils.data.DataLoader(val_dataset, shuffle=True, batch_size=4)

# Huấn luyện mô hình

In [ ]:
from tqdm import tqdm
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

best_val_loss = float('inf')
best_model_path = "best_model.pth"
epochs_no_improve = 0
n_epochs_stop = 2
max_epochs = 30

for epoch in range(max_epochs):
    print(f"Epoch {epoch + 1}/{max_epochs}")

    # Training Phase
    model.train()
    epoch_train_loss = 0
    with tqdm(train_dataloader, desc=f"Epoch {epoch + 1} (Train)", unit="batch") as tepoch:
        for idx, batch in enumerate(tepoch):
            input_ids = batch.pop("input_ids").to(device)
            pixel_values = batch.pop("pixel_values").to(device)

            outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            labels=input_ids)

            loss = outputs.loss
            epoch_train_loss += loss.item()

            tepoch.set_postfix(loss=loss.item())

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    avg_train_loss = epoch_train_loss / len(train_dataloader)
    print(f"Average training loss for epoch {epoch + 1}: {avg_train_loss:.4f}")

    # Validation Phase
    model.eval()
    epoch_val_loss = 0
    with torch.no_grad():
        with tqdm(val_dataloader, desc=f"Epoch {epoch + 1} (Val)", unit="batch") as vepoch:
            for idx, batch in enumerate(vepoch):
                input_ids = batch.pop("input_ids").to(device)
                pixel_values = batch.pop("pixel_values").to(device)

                outputs = model(input_ids=input_ids,
                                pixel_values=pixel_values,
                                labels=input_ids)

                loss = outputs.loss
                epoch_val_loss += loss.item()

                vepoch.set_postfix(loss=loss.item())

    avg_val_loss = epoch_val_loss / len(val_dataloader)
    print(f"Average validation loss for epoch {epoch + 1}: {avg_val_loss:.4f}")

    # Save the best model and check for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved to {best_model_path} with validation loss: {best_val_loss:.4f}")
        epochs_no_improve = 0  # Reset lại số epoch không cải thiện
    else:
        epochs_no_improve += 1
        print(f"Validation loss did not improve for {epochs_no_improve} epochs.")

    # Early stopping check
    if epochs_no_improve == n_epochs_stop:
        print(f"Early stopping triggered after {epoch + 1} epochs.")
        break

print("Training finished.")

# Đánh giá mô hình

In [ ]:
df_test

,image_path,caption
0,/dataset/test/images/000000205086.jpg,Một cậu bé đang chống đẩy trên quả banh.\n
1,/dataset/test/images/000000205086.jpg,Một cậu bé đang chống đầu lên quả bóng đá hít ...
2,/dataset/test/images/000000205086.jpg,Một cậu bé đang hít đất đầu trên quả banh trên...
3,/dataset/test/images/000000205086.jpg,Một cậu bé đang hít đất cùng quả bóng và một c...
4,/dataset/test/images/000000205086.jpg,Một cậu bé đang chống đẩy trên một quả bóng đá.\n
...,...,...
1150,/dataset/test/images/000000509030.jpg,Đứa trẻ đang chơi bóng chày ngoài sân .\n
1151,/dataset/test/images/000000554036.jpg,Một cầu thủ bóng chày cầm gậy trên sân bóng ch...
1152,/dataset/test/images/000000014864.jpg,Hai bức ảnh của vận động viên tennis nữ đang d...
1153,/dataset/test/images/000000463758.jpg,Hai bức ảnh của một người đàn ông đang chơi te...


In [ ]:
df_test['caption'].iloc[1]

'Một cậu bé đang chống đầu lên quả bóng đá hít đất.\n'

In [ ]:
def text_preprocessing_test(text):
    text = text.replace('\n', ' ')
    return text

In [ ]:
df_test['caption'] = df_test['caption'].apply(lambda x: text_preprocessing_test(x))

In [ ]:
df_test = df_test.groupby('image_path')['caption'].agg(list).reset_index()
df_test.rename(columns={'caption': 'caption'}, inplace=True)

df_test['caption'].iloc[0]

['Ở trên sân , một cầu thủ đánh bóng đang chuẩn bị đánh bóng . ',
 'Ở trên sân , một cầu thủ đánh bóng đang luyện tập . ',
 'Ở trên sân , một cầu thủ đánh bóng đang vung gậy để đánh bóng . ',
 'Ở trên sân , một người đàn ông đang nghiêng người để chuẩn bị đánh quả bóng chày . ',
 'Một người đàn ông đang vung gậy để đánh quả bóng chày . ']

In [ ]:
df_test

,image_path,caption
0,/dataset/test/images/000000007615.jpg,"[Ở trên sân , một cầu thủ đánh bóng đang chuẩn..."
1,/dataset/test/images/000000009895.jpg,[Hình ảnh hai cậu bé đang chơi trong khu dân c...
2,/dataset/test/images/000000011015.jpg,[Hình ảnh một trận thi đấu bóng đá đang diễn r...
3,/dataset/test/images/000000014864.jpg,[Một người phụ nữ đang vung vợt để đánh quả bó...
4,/dataset/test/images/000000014876.jpg,[Một cậu bé đang nghiêng người để đánh quả bón...
...,...,...
226,/dataset/test/images/000000562596.jpg,[Một người đàn ông đang dùng vợt để đánh quả b...
227,/dataset/test/images/000000564627.jpg,[Một người đàn ông đang chuẩn bị đánh quả bóng...
228,/dataset/test/images/000000565360.jpg,[Các cầu thủ bóng đá đang thi đấu ở ngoài trời...
229,/dataset/test/images/000000572998.jpg,[Một cầu thủ đánh bóng đang xoay người để đánh...


In [ ]:
test_dataset = Dataset.from_pandas(df_test)
test_dataset = test_dataset.map(lambda example: {"image": load_image(example["image_path"])}, batched=False)

Map:   0%|          | 0/231 [00:00<?, ? examples/s]

In [ ]:
from tqdm import tqdm

for index, record in tqdm(df_test.iterrows(), total=len(df_test), desc="Processing Captions"):
    img = test_dataset[index]['image']
    inputs = processor(images=img, return_tensors="pt").to(device)
    pixel_values = inputs.pixel_values
    generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    caption = ' '.join(generated_caption.replace('_', '').split())
    df_test.loc[index, 'predict'] = caption


Processing Captions: 100%|██████████| 231/231 [03:42<00:00,  1.04it/s]


## Hiển thị predict caption

Ở đây mô hình không đưa ra câu dự đoán nên không thể đánh giá được


In [ ]:
df_test

,image_path,caption,predict
0,/dataset/test/images/000000007615.jpg,"[Ở trên sân , một cầu thủ đánh bóng đang chuẩn...",
1,/dataset/test/images/000000009895.jpg,[Hình ảnh hai cậu bé đang chơi trong khu dân c...,
2,/dataset/test/images/000000011015.jpg,[Hình ảnh một trận thi đấu bóng đá đang diễn r...,
3,/dataset/test/images/000000014864.jpg,[Một người phụ nữ đang vung vợt để đánh quả bó...,
4,/dataset/test/images/000000014876.jpg,[Một cậu bé đang nghiêng người để đánh quả bón...,
...,...,...,...
226,/dataset/test/images/000000562596.jpg,[Một người đàn ông đang dùng vợt để đánh quả b...,
227,/dataset/test/images/000000564627.jpg,[Một người đàn ông đang chuẩn bị đánh quả bóng...,
228,/dataset/test/images/000000565360.jpg,[Các cầu thủ bóng đá đang thi đấu ở ngoài trời...,
229,/dataset/test/images/000000572998.jpg,[Một cầu thủ đánh bóng đang xoay người để đánh...,
